In [1]:
!git clone https://github.com/yunsuxiaozi/Yunbase.git

Cloning into 'Yunbase'...
remote: Enumerating objects: 250, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 250 (delta 58), reused 69 (delta 30), pack-reused 150 (from 1)
Receiving objects: 100% (250/250), 552.60 KiB | 4.49 MiB/s, done.
Resolving deltas: 100% (140/140), done.


In [2]:
!pip  download -r Yunbase/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.5/164.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.0/36.0 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [3]:
!pip install -q --requirement Yunbase/requirements.txt  \
--no-index --find-links file:.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cesium 0.12.3 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
tsfresh 0.20.3 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.


In [4]:
from Yunbase.baseline import Yunbase

#necessary
import polars as pl#similar to pandas, but with better performance when dealing with large datasets.
import pandas as pd#read csv,parquet
import numpy as np#for scientific computation of matrices
#models(lgb,xgb,cat)
from  lightgbm import LGBMRegressor,LGBMClassifier,log_evaluation,early_stopping
from catboost import CatBoostRegressor,CatBoostClassifier
from xgboost import XGBRegressor,XGBClassifier
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer#word2vec feature
import warnings#avoid some negligible errors
#The filterwarnings () method is used to set warning filters, which can control the output method and level of warning information.
warnings.filterwarnings('ignore')

import random#provide some function to generate random_seed.
#set random seed,to make sure model can be recurrented.
def seed_everything(seed):
    np.random.seed(seed)#numpy's random seed
    random.seed(seed)#python built-in random seed
seed_everything(seed=2024)

In [5]:
train=pl.read_csv("/kaggle/input/brist1d/train.csv")
train=train.to_pandas()
test=pl.read_csv("/kaggle/input/brist1d/test.csv")
test=test.to_pandas()
print(len(train),len(test))
train.head()

177024 3644


,id,p_num,time,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,...,activity-0:40,activity-0:35,activity-0:30,activity-0:25,activity-0:20,activity-0:15,activity-0:10,activity-0:05,activity-0:00,bg+1:00
0,p01_0,p01,06:10:00,None,NaN,9.6,None,NaN,9.7,None,...,None,None,None,None,None,None,None,None,None,13.4
1,p01_1,p01,06:25:00,None,NaN,9.7,None,NaN,9.2,None,...,None,None,None,None,None,None,None,None,None,12.8
2,p01_2,p01,06:40:00,None,NaN,9.2,None,NaN,8.7,None,...,None,None,None,None,None,None,None,None,None,15.5
3,p01_3,p01,06:55:00,None,NaN,8.7,None,NaN,8.4,None,...,None,None,None,None,None,None,None,None,None,14.8
4,p01_4,p01,07:10:00,None,NaN,8.4,None,NaN,8.1,None,...,None,None,None,None,None,None,None,None,None,12.7


In [6]:
#six float features
feats=['bg','insulin','hr','steps','cals','activity']
ts=['-5:55', '-5:50', '-5:45', '-5:40', '-5:35', '-5:30', '-5:25', '-5:20', '-5:15', '-5:10', '-5:05', '-5:00', '-4:55', '-4:50', '-4:45', '-4:40', '-4:35', '-4:30', '-4:25', '-4:20', '-4:15', '-4:10', '-4:05', '-4:00', '-3:55', '-3:50', '-3:45', '-3:40', '-3:35', '-3:30', '-3:25', '-3:20', '-3:15', '-3:10', '-3:05', '-3:00', '-2:55', '-2:50', '-2:45', '-2:40', '-2:35', '-2:30', '-2:25', '-2:20', '-2:15', '-2:10', '-2:05', '-2:00', '-1:55', '-1:50', '-1:45', '-1:40', '-1:35', '-1:30', '-1:25', '-1:20', '-1:15', '-1:10', '-1:05', '-1:00', '-0:55', '-0:50', '-0:45', '-0:40', '-0:35', '-0:30', '-0:25', '-0:20', '-0:15', '-0:10', '-0:05', '-0:00']
def get_more_data(df,timestep,mode='train'):#mode is train or test
    full_data=[]
    #12step is 1 hour.
    for start in range(0,len(ts)-timestep-12):
        end=start+timestep
        target=end+12
        #[start:end] target
        hours_data=df[['p_num','time']+[f+ts[i]  for i in range(start,end) for f in feats]+['bg'+ts[target]]]
        hours_data.columns=['p_num','time']+[f+str(i-start)  for i in range(start,end)for f in feats]+['bg+1:00']
        hours_data=hours_data[~hours_data['bg+1:00'].isna()]
        #get true time
        hours_data['time']=hours_data['time']-pd.DateOffset(minutes=(len(ts)-end)*5)
        hours_data['hour']=hours_data['time'].dt.hour
        hours_data['minute']=hours_data['time'].dt.minute
        hours_data['sin_hour']=np.sin(2*np.pi*hours_data['hour']/24)
        hours_data['cos_hour']=np.cos(2*np.pi*hours_data['hour']/24)
        hours_data['sin_halfday']=np.sin(2*np.pi*hours_data['hour']/12)
        hours_data['cos_halfday']=np.cos(2*np.pi*hours_data['hour']/12)
        
        full_data.append(hours_data)
    if mode=='test':
        hours_data=df[['p_num','time']+[f+ts[i]  for i in range(len(ts)-timestep,len(ts)) for f in feats]]
        hours_data.columns=['p_num','time']+[f+str(i-len(ts)+timestep)  for i in range(len(ts)-timestep,len(ts))for f in feats]
        #get true time
        hours_data['time']=hours_data['time']
        hours_data['hour']=hours_data['time'].dt.hour
        hours_data['minute']=hours_data['time'].dt.minute
        hours_data['sin_hour']=np.sin(2*np.pi*hours_data['hour']/24)
        hours_data['cos_hour']=np.cos(2*np.pi*hours_data['hour']/24)
        hours_data['sin_halfday']=np.sin(2*np.pi*hours_data['hour']/12)
        hours_data['cos_halfday']=np.cos(2*np.pi*hours_data['hour']/12)
        
        full_data.append(hours_data)
    full_data=pd.concat(full_data).drop_duplicates()
    return full_data.drop(['time'],axis=1)

In [7]:
def FE(df,mode='train',timestep=9):#mode='train' or 'test'
    #mean_isnan is 0.98 ,id is useless.
    print("< drop useless columns >")
    df.drop([c for c in df.columns if 'carbs' in c]+['id'],axis=1,inplace=True)

    #nearest bgs' correlation is greater than 0.99.
    print("< fillna >")
    bgs=[c for c in list(df.columns) if (c!='bg+1:00') and ('bg' in c)]
    df[bgs] = df[bgs].fillna(method='ffill', axis=1)
    df[bgs] = df[bgs].fillna(method='bfill', axis=1)

    #long tail distribution
    print("< log1p transform >")
    step_cols=[c for c in list(df.columns) if 'step' in c]
    df[step_cols]=df[step_cols].fillna(50).astype(np.float32)
    df[step_cols]=np.log1p(df[step_cols])

    #transform None to np.nan.
    print("< fix bug >")
    float_cols=[c for c in df.columns  for f in feats[:-1] if f in c]
    df[float_cols]=df[float_cols].astype(np.float32)

    print("< get more data >")
    df['time']=pd.to_datetime(df['time'])
    df=get_more_data(df,timestep=timestep,mode=mode)
    #p_num to int
    df['p_num_int']=df['p_num'].apply(lambda x:int(x[1:])).astype(np.int8)
    
    float_cols=[col+str(ts) for ts in range(timestep) for col in ['bg','insulin','hr','steps','cals']]
    df[float_cols]=df[float_cols].fillna(-1)

    #activity to str then drop them.
    print("< activity to str >")
    activity_cols=[c for c in list(df.columns) if 'activity' in c]
    activity_feats=df[activity_cols].astype(str).replace('None','').values
    df['activity_str']=[' '.join(activity_feats[i]) for i in range(len(activity_feats))]

    print("< Z-score feature >")
    cols=['bg', 'insulin','steps','hr','cals']
    for col in cols:
        df[f'mean_pnum_{col}{timestep-1}']=df.groupby(['p_num'])[col+f"{timestep-1}"].transform('mean')
        df[f'std_pnum_{col}{timestep-1}']=df.groupby(['p_num'])[col+f"{timestep-1}"].transform('std')
        df[f'norm_pnum_{col}{timestep-1}']=(df[f'{col}{timestep-1}']-df[f'mean_pnum_{col}{timestep-1}'])/df[f'std_pnum_{col}{timestep-1}']
        df[f'{col}{timestep-1}-{col}{timestep-2}']=df[f'{col}{timestep-1}']-df[f'{col}{timestep-2}']
        df[f'{col}{timestep-1}+{col}{timestep-2}']=df[f'{col}{timestep-1}']+df[f'{col}{timestep-2}']
        df[f'{col}{timestep-1}*{col}{timestep-2}']=df[f'{col}{timestep-1}']*df[f'{col}{timestep-2}']
        df[f'{col}{timestep-1}/{col}{timestep-2}']=df[f'{col}{timestep-1}']/df[f'{col}{timestep-2}']
    print("< cross feature >")
    cross=['bg', 'insulin','steps','hr','cals']
    for i in range(len(cross)):
        for j in range(i+1,len(cross)):
            for k in [1,2,3]:
                df[cross[i]+f"{timestep-k}+"+cross[j]+f"{timestep-k}"]=df[cross[i]+f"{timestep-k}"]+df[cross[j]+f"{timestep-k}"]
                df[cross[i]+f"{timestep-k}-"+cross[j]+f"{timestep-k}"]=df[cross[i]+f"{timestep-k}"]-df[cross[j]+f"{timestep-k}"]
                df[cross[i]+f"{timestep-k}*"+cross[j]+f"{timestep-k}"]=df[cross[i]+f"{timestep-k}"]*df[cross[j]+f"{timestep-k}"]
                df[cross[i]+f"{timestep-k}/"+cross[j]+f"{timestep-k}"]=df[cross[i]+f"{timestep-k}"]/(df[cross[j]+f"{timestep-k}"]+1e-15)
            df[cross[i]+f"{timestep-1}-"+cross[j]+f"{timestep-1}-"+cross[i]+f"{timestep-2}-"+cross[j]+f"{timestep-2}"]=\
            df[cross[i]+f"{timestep-1}-"+cross[j]+f"{timestep-1}"]-df[cross[i]+f"{timestep-2}-"+cross[j]+f"{timestep-2}"]
    
    
    for col in ['bg', 'insulin','steps','hr','cals']:
        for i in range(1,timestep):
            df[col+f"{i}"+f"-{col}{i-1}"]=df[col+f"{i}"]-df[col+f"{i-1}"]
    
    #strong correlation between nearest columns
    df.drop([f'bg{i}'for i in range(timestep) if i%2!=0],axis=1,inplace=True)
    return df

train=FE(train,mode='train')
test=FE(test,mode='test')
total=pd.concat((train,test))
print(len(train),len(test),len(total))
test=total[total['bg+1:00'].isna()]
train=total[~total['bg+1:00'].isna()]
print(len(train),len(test),len(total))
train.head()

< drop useless columns >
< fillna >
< log1p transform >
< fix bug >
< get more data >
< activity to str >
< Z-score feature >
< cross feature >
< drop useless columns >
< fillna >
< log1p transform >
< fix bug >
< get more data >
< activity to str >
< Z-score feature >
< cross feature >
309438 189488 498926
495282 3644 498926


,p_num,bg0,insulin0,hr0,steps0,cals0,activity0,insulin1,hr1,steps1,...,hr5-hr4,hr6-hr5,hr7-hr6,cals1-cals0,cals2-cals1,cals3-cals2,cals4-cals3,cals5-cals4,cals6-cals5,cals7-cals6
0,p01,9.6,0.0083,-1.0,3.931826,-1.0,None,0.0083,-1.0,3.931826,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,p01,9.7,0.0083,-1.0,3.931826,-1.0,None,0.0083,-1.0,3.931826,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,p01,9.2,0.0083,-1.0,3.931826,-1.0,None,0.0083,-1.0,3.931826,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,p01,8.7,0.0083,-1.0,3.931826,-1.0,None,0.0083,-1.0,3.931826,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,p01,8.4,0.0083,-1.0,3.931826,-1.0,None,0.0083,-1.0,3.931826,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
yunbase=Yunbase(num_folds=10,
                  models=[],
                  FE=None,
                  drop_cols=[],
                  seed=2024,
                  objective='regression',
                  metric='rmse',
                  nan_margin=0.95,
                  group_col='p_num',
                  target_col='bg+1:00',
                  device='gpu',
                  early_stop=250,
                  use_high_corr_feat=False,
                  text_cols=['activity_str'],
                  word2vec_models=[(TfidfVectorizer(max_features=150,ngram_range=(1,2)),'activity_str','tfidf'),
                                   (CountVectorizer(max_features=150,ngram_range=(3,4)),'activity_str','wordcnt')
                                  ],
               )
category_cols=['activity0', 'activity1', 'activity2', 'activity3', 'activity4', 'activity5', 'activity6', 'activity7', 'activity8']
yunbase.fit(train_path_or_file=train,category_cols=category_cols)
test_preds=yunbase.predict(test_path_or_file=test,weights=[5,2.5,2.5])
yunbase.submit(submission_path_or_file='/kaggle/input/brist1d/sample_submission.csv',test_preds=test_preds)

Currently supported metrics:['custom_metric', 'mae', 'rmse', 'mse', 'medae', 'rmsle', 'msle', 'mape', 'r2', 'auc', 'logloss', 'f1_score', 'mcc', 'accuracy', 'multi_logloss']
Currently supported models:['lgb', 'cat', 'xgb', 'ridge', 'LinearRegression', 'LogisticRegression', 'tabnet']
Currently supported kfolds:['KFold', 'GroupKFold', 'StratifiedKFold', 'StratifiedGroupKFold', 'purged_CV']
Currently supported objectives:['binary', 'multi_class', 'regression']
fit......
load train data
train.shape:(458035, 260)
Feature Engineer
< text column's feature >
-> for column activity_str word feature
-> for column activity_str sentence feature
-> for column activity_str paragraph feature
< one hot encoder >
-> for column activity0 labelencoder feature
-> for column activity1 labelencoder feature
-> for column activity2 labelencoder feature
-> for column activity3 labelencoder feature
-> for column activity4 labelencoder feature
-> for column activity5 labelencoder feature
-> for column activity6 

1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


Training until validation scores don't improve for 250 rounds
[100]	valid_0's rmse: 2.02892
[200]	valid_0's rmse: 2.02594
[300]	valid_0's rmse: 2.02777
[400]	valid_0's rmse: 2.02892
Early stopping, best iteration is:
[168]	valid_0's rmse: 2.02355
name:lgb,fold:1
< word2vec >
-> for column activity_str tfidf word2vec feature
-> for column activity_str wordcnt word2vec feature
< word2vec >
-> for column activity_str tfidf word2vec feature
-> for column activity_str wordcnt word2vec feature
Training until validation scores don't improve for 250 rounds
[100]	valid_0's rmse: 2.2949
[200]	valid_0's rmse: 2.29575
[300]	valid_0's rmse: 2.30203
Early stopping, best iteration is:
[130]	valid_0's rmse: 2.29149
name:lgb,fold:2
< word2vec >
-> for column activity_str tfidf word2vec feature
-> for column activity_str wordcnt word2vec feature
< word2vec >
-> for column activity_str tfidf word2vec feature
-> for column activity_str wordcnt word2vec feature
Training until validation scores don't improv